## ※商品マスタのCSVを取込み、Silverとして扱う

In [0]:
%run /Workspace/Users/takahiro.koizumi@nsw.co.jp/EX_error_log_export

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import col
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import lit
from datetime import datetime, timedelta

In [0]:
# Spark session作成
def spark_session(app_name):
    print("Spark session 作成中…")
    try:
        spark = SparkSession.builder.appName(app_name).getOrCreate()
        print("Spark session 作成完了")
        return spark

    # エラー処理
    except Exception as e:
        # ログを出力する
        logger = setup_logging(logtype="error")
        logger.error(f"Spark session 作成中にエラーが発生しました: {e}")
        return False

In [0]:
# csvのDataFrame化
def csv_to_dataframe(spark, csv_file, temp_view_name):
    print("CSVファイルをDataFrameに変換中…")
    try:
        # 各CSVファイルを読み込み、DataFrameを作成
        csv_df = spark.read.csv(csv_file, header=True, inferSchema=True,nullValue="NULL")
        
        # 一時ビューを作成
        temp_view = csv_df.createOrReplaceTempView(temp_view_name)
        
        # レコード数（行数）を取得
        row_count_csv = csv_df.count()
        print(f"変換完了: {row_count_csv}件")
        return True

    # エラー処理
    except Exception as e:
        logger = setup_logging(logtype="error")
        logger.error(f"Error: {e}")
        print(f"csvファイルの変換に失敗しました。エラー: {e}")
        return False

In [0]:
# Silverテーブルを作成
def create_table(Silver_table_name):
    print(f"{Silver_table_name}の空テーブルを作成中…")
    try:
        spark.sql(
            f"""
        CREATE TABLE { Silver_table_name } (
            ID varchar(30) NOT NULL,
            COMPANY_CODE varchar(2) NOT NULL,
            MODEL_NO varchar(40),
            PRODUCT_CODE varchar(24) NOT NULL,
            PRODUCT_NAME varchar(20),
            SERIES_NAME varchar(100),
            JAN_CODE varchar(13),
            COLOR_CODE varchar(5),
            LARGE_CLASS_CODE varchar(5),
            LARGE_CLASS_NAME varchar(20),
            MIDDLE_CLASS_CODE varchar(60),
            MIDDLE_CLASS_NAME varchar(60),
            SMALL_CLASS_CODE varchar(10),
            SMALL_CLASS_NAME varchar(20),
            LENS_SAHPE_CODE varchar(100),
            LENS_SAHPE_NAME varchar(100),
            PO_CATEGORY_CODE varchar(2),
            PO_CATEGORY_NAME varchar(50),
            RELEASE_DATE timestamp,
            LAST_RECEIPT_DATE timestamp,
            FRAME_HEIGHT decimal(8, 2),
            LENS_WIDTH decimal(8, 2),
            TARGET_CODE varchar(5),
            TARGET_NAME varchar(12),
            REPEAT_COUNT decimal(8, 2),
            DISCONTINUED_FLAG varchar(5),
            FIRST_SELLING_PRICE decimal(8, 2),
            SELLING_PRICE_PROPER decimal(8, 2),
            SELLING_PRICE_OUTLET decimal(8, 2),
            ACCOUT_PATTERN_CODE varchar(5),
            ACCOUT_PATTERN_NAME varchar(60),
            ACCOUNT_TITLE_CODE varchar(5),
            ACCOUNT_TITLE_NAME varchar(60),
            PRODUCT_CLASS varchar(5),
            SUBSIDIARY_ACCOUNT_CODE varchar(5),
            SUBSIDIARY_ACCOUNT_NAME varchar(60),
            MANUFUCTURER_CODE varchar(10),
            FACTORY_CODE varchar(10),
            BUDGET_ACTUAL_CODE varchar(5),
            BUDGET_ACTUAL_NAME varchar(50),
            SECTION_CODE varchar(5),
            SECTION_NAME varchar(50),
            COATING_CODE varchar(5),
            COATING_NAME varchar(50),
            FUNCTION_CODE varchar(5),
            FUNCTION_NAME varchar(50),
            DESIGN_CODE varchar(5),
            DESIGN_NAME varchar(50),
            HEIGHT_MM varchar(50),
            WIDTH_MM varchar(50),
            SALETYPE varchar(50),
            PRIMARY KEY (ID, COMPANY_CODE, PRODUCT_CODE)
        ) USING DELTA
        """
        )
        print(f"作成完了")
        return True

    # エラー処理
    except Exception as e:
        logger = setup_logging(logtype="error")
        logger.error(f"Error: {e}")
        print(f"csvファイルの変換に失敗しました。エラー: {e}")
        return False

In [0]:
# 一時ビューから新しいテーブルにデータを挿入
def insert_data(silver_table_name):
    print(f"{silver_table_name}にデータを挿入中…")
    try:
        spark.sql(
            f"""
        INSERT INTO {silver_table_name}
            SELECT
                ID,
                COMPANY_CODE,
                MODEL_NO,
                PRODUCT_CODE,
                PRODUCT_NAME,
                SERIES_NAME,
                JAN_CODE,
                COLOR_CODE,
                LARGE_CLASS_CODE,
                LARGE_CLASS_NAME,
                MIDDLE_CLASS_CODE,
                MIDDLE_CLASS_NAME,
                SMALL_CLASS_CODE,
                SMALL_CLASS_NAME,
                LENS_SAHPE_CODE,
                LENS_SAHPE_NAME,
                PO_CATEGORY_CODE,
                PO_CATEGORY_NAME,
                RELEASE_DATE,
                LAST_RECEIPT_DATE,
                FRAME_HEIGHT,
                LENS_WIDTH,
                TARGET_CODE,
                TARGET_NAME,
                REPEAT_COUNT,
                DISCONTINUED_FLAG,
                FIRST_SELLING_PRICE,
                SELLING_PRICE_PROPER,
                SELLING_PRICE_OUTLET,
                ACCOUT_PATTERN_CODE,
                ACCOUT_PATTERN_NAME,
                ACCOUNT_TITLE_CODE,
                ACCOUNT_TITLE_NAME,
                PRODUCT_CLASS,
                SUBSIDIARY_ACCOUNT_CODE,
                SUBSIDIARY_ACCOUNT_NAME,
                MANUFUCTURER_CODE,
                FACTORY_CODE,
                BUDGET_ACTUAL_CODE,
                BUDGET_ACTUAL_NAME,
                SECTION_CODE,
                SECTION_NAME,
                COATING_CODE,
                COATING_NAME,
                FUNCTION_CODE,
                FUNCTION_NAME,
                DESIGN_CODE,
                DESIGN_NAME,
                HEIGHT_MM,
                WIDTH_MM,
                SALETYPE
            FROM {temp_view_name}
        """
        )
        row_count_insert = spark.read.table(silver_table_name).count()
        print(f"挿入完了 　総データ件数:{row_count_insert}件")
        return True
    
    # エラー処理
    except Exception as e:
        logger = setup_logging(logtype="error")
        logger.error(f"Error: {e}")
        print(f"Silverテーブル作成に失敗しました。エラー: {e}")
        return False

In [0]:
# 作成したSilverテーブルの全件削除
def delete_all_data(silver_table_name):
    print(f"{silver_table_name}の全データを削除中…")
    try:
        spark.sql(
            f"""
        TRUNCATE TABLE {silver_table_name}
        """
        )
        print(f"削除完了")
        return True
    except Exception as e:
        logger = setup_logging(logtype="error")
        logger.error(f"Error: {e}")
        print(f"削除に失敗しました。エラー: {e}")
        return False

In [0]:
# 　実行関数
def silver_layer(csv_file, temp_view_name, silver_table_name):
    # Silverテーブルが存在しない場合
    if not spark.catalog.tableExists(silver_table_name):
        print(f"{silver_table_name}が存在しないため新規作成します。")
        # sppark session起動
        spark_session(app_name)

        # csvファイルをDataFrameに変換
        csv_to_dataframe(spark, csv_file, temp_view_name)
        
        # Silverテーブルを作成
        create_table(silver_table_name)
        
        # Silverテーブルにデータを挿入
        insert_data(silver_table_name)
        return True

    # Silverテーブルが存在する場合
    else:
        print(f"{silver_table_name}が存在するので全件入替を実施します。")
        # sppark session起動
        spark_session(app_name)
        
        # 既存レコードの全削除
        delete_all_data(silver_table_name)

        # csvファイルをDataFrameに変換
        csv_to_dataframe(spark, csv_file, temp_view_name)

        # Silverテーブルにデータを挿入
        insert_data(silver_table_name)
        return True
    

In [0]:
# 昨日日付を取得する
def generate_csv_path_yesterday(base_path: str, prefix: str) -> str:
    yesterday = (datetime.today() - timedelta(days=1)).strftime("%Y%m%d")
    return f"{base_path}/{prefix}_{yesterday}.csv"

In [0]:
# app_name を指定してください。
app_name = "Silver_layer"
# ディレクトリパス
base_path ="/FileStore/tables"
# テーブル種類（マスタ名）
prefix = "product"
# 完成したCSVファイルのパス（昨日日付）
csv_file = [generate_csv_path_yesterday(base_path, prefix)]

# 一時ビュー名を指定してください。
temp_view_name = "temp_product"
# Silverテーブル名を入力してください。
silver_table_name = "prd_im_dlh.silver.dim_product"

# 実行
silver_layer(csv_file, temp_view_name, silver_table_name)